In [1]:
import os
import ast
import sys
import json
import torch
import pickle
import subprocess

sys.path.append('../')

import pandas as pd

from pathlib import Path
from tqdm import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report
#from utils.post_processing import post_process


In [2]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

In [3]:
CURRENT_DIR = Path.cwd()
FT_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning"
DATASET_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning" / "datasets"

ERC_DIR = FT_DIR.parent
LLAMA_FACTORY_DIR = ERC_DIR / "LLaMA-Factory"

BASE_MODEL = "unsloth/Qwen2.5-32B-Instruct-bnb-4bit"
LOGGING_DIR = FT_DIR / "training_logs"
OUTPUT_DIR = FT_DIR / "saved_models" / f"""comics35_pg_nb_{BASE_MODEL.split("/")[1]}"""

In [4]:
DATASET_DIR = CURRENT_DIR / "emotion_analysis_comics" / "finetuning" / "datasets"

In [5]:
train_dataset_name = f"""comics35_utterance_pg_train.json"""
test_dataset_name = f"""comics35_utterance_pg_test.json"""

train_dataset_file = DATASET_DIR / train_dataset_name
test_dataset_file = DATASET_DIR / test_dataset_name

In [6]:
train_dataset_file

PosixPath('/Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/datasets/comics35_utterance_pg_train.json')

In [7]:

if not os.path.exists(os.path.join(FT_DIR, "model_args")):
    os.mkdir(os.path.join(FT_DIR, "model_args"))

train_file = FT_DIR / "model_args" / f"""{train_dataset_name.split(".")[0].split("train")[0]}{BASE_MODEL.split("/")[1]}.json"""

In [8]:
dataset_info_line =  {
  "file_name": f"{train_dataset_file}",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output"
  }
}

In [9]:
with open(os.path.join(LLAMA_FACTORY_DIR, "data/dataset_info.json"), "r") as jsonFile:
    data = json.load(jsonFile)

data["comics"] = dataset_info_line

with open(os.path.join(LLAMA_FACTORY_DIR, "data/dataset_info.json"), "w") as jsonFile:
    json.dump(data, jsonFile)

In [10]:
NB_EPOCHS = 8

In [11]:
args = dict(
    
  stage="sft",                           # do supervised fine-tuning
  do_train=True,

  model_name_or_path=BASE_MODEL,         # use bnb-4bit-quantized Llama-3-8B-Instruct model
  num_train_epochs=NB_EPOCHS,            # the epochs of training
  output_dir=str(OUTPUT_DIR),                 # the path to save LoRA adapters
  overwrite_output_dir=True,             # overrides existing output contents

  dataset="comics",                      # dataset name
  template="qwen",                     # use llama3 prompt template
  #train_on_prompt=True,
  val_size=0.1,
  max_samples=10000,                       # use 500 examples in each dataset

  finetuning_type="lora",                # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  per_device_train_batch_size=2,         # the batch size
  gradient_accumulation_steps=4,         # the gradient accumulation steps
  lr_scheduler_type="cosine",            # use cosine learning rate scheduler
  loraplus_lr_ratio=16.0,                # use LoRA+ algorithm with lambda=16.0
  #temperature=0.5,
  
  warmup_ratio=0.1,                      # use warmup scheduler    
  learning_rate=5e-5,                    # the learning rate
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  
  fp16=True,                             # use float16 mixed precision training
  quantization_bit=4,                    # use 4-bit QLoRA  
  #use_liger_kernel=True,
  #quantization_device_map="auto",
  
  logging_steps=10,                      # log every 10 steps
  save_steps=5000,                       # save checkpoint every 1000 steps    
  logging_dir=str(LOGGING_DIR),
  
  # use_unsloth=True,
  report_to="tensorboard"                       # discards wandb

)

In [12]:
json.dump(args, open(train_file, "w", encoding="utf-8"), indent=2)

In [13]:
p = subprocess.Popen(["llamafactory-cli", "train", train_file], cwd=LLAMA_FACTORY_DIR)

In [14]:
p.wait()

11/08/2024 13:13:13 - INFO - llamafactory.cli - Initializing distributed tasks at: 127.0.0.1:29878


W1108 13:13:14.620000 140594776962368 torch/distributed/run.py:779] 
W1108 13:13:14.620000 140594776962368 torch/distributed/run.py:779] *****************************************
W1108 13:13:14.620000 140594776962368 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1108 13:13:14.620000 140594776962368 torch/distributed/run.py:779] *****************************************


11/08/2024 13:13:31 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
11/08/2024 13:13:31 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
11/08/2024 13:13:31 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, compute dtype: torch.float16


[INFO|configuration_utils.py:672] 2024-11-08 13:13:31,690 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2.5-32B-Instruct-bnb-4bit/snapshots/36adcc220ce3f5747a2ac662a8c11e7236aedc27/config.json
[INFO|configuration_utils.py:739] 2024-11-08 13:13:31,696 >> Model config Qwen2Config {
  "_name_or_path": "unsloth/Qwen2.5-32B-Instruct-bnb-4bit",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 27648,
  "max_position_embeddings": 32768,
  "max_window_layers": 70,
  "model_type": "qwen2",
  "num_attention_heads": 40,
  "num_hidden_layers": 64,
  "num_key_value_heads": 8,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bi

11/08/2024 13:13:32 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
11/08/2024 13:13:32 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
11/08/2024 13:13:32 - INFO - llamafactory.hparams.parser - Process rank: 3, device: cuda:3, n_gpu: 1, distributed training: True, compute dtype: torch.float16
11/08/2024 13:13:32 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
11/08/2024 13:13:32 - WARNING - llamafactory.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
11/08/2024 13:13:32 - INFO - llamafactory.hparams.parser - Process rank: 4, device: cuda:4, n_gpu: 1, distributed training: True, compute dtype: torch.float16
11/08/2024 13:13:32 - WARNING - llamafactory.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.
11/08/2024 13:13:32 - WARN

...

{'train_runtime': 2427.1046, 'train_samples_per_second': 2.129, 'train_steps_per_second': 0.053, 'train_loss': 0.2546650299336761, 'epoch': 7.88}


[INFO|configuration_utils.py:672] 2024-11-08 13:54:56,680 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2.5-32B-Instruct-bnb-4bit/snapshots/36adcc220ce3f5747a2ac662a8c11e7236aedc27/config.json
[INFO|configuration_utils.py:739] 2024-11-08 13:54:56,681 >> Model config Qwen2Config {
  "_name_or_path": "Qwen/Qwen2.5-32B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 27648,
  "max_position_embeddings": 32768,
  "max_window_layers": 70,
  "model_type": "qwen2",
  "num_attention_heads": 40,
  "num_hidden_layers": 64,
  "num_key_value_heads": 8,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type

***** train metrics *****
  epoch                    =      7.8769
  total_flos               = 507156060GF
  train_loss               =      0.2547
  train_runtime            =  0:40:27.10
  train_samples_per_second =       2.129
  train_steps_per_second   =       0.053


[INFO|modelcard.py:449] 2024-11-08 13:55:01,936 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}


0

In [15]:
args = dict(
  model_name_or_path=BASE_MODEL, # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path=str(OUTPUT_DIR),            # load the saved LoRA adapters
  template="qwen",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)

In [16]:
model = ChatModel(args)

[INFO|configuration_utils.py:672] 2024-11-08 13:55:04,124 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2.5-32B-Instruct-bnb-4bit/snapshots/36adcc220ce3f5747a2ac662a8c11e7236aedc27/config.json
[INFO|configuration_utils.py:739] 2024-11-08 13:55:04,130 >> Model config Qwen2Config {
  "_name_or_path": "unsloth/Qwen2.5-32B-Instruct-bnb-4bit",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 27648,
  "max_position_embeddings": 32768,
  "max_window_layers": 70,
  "model_type": "qwen2",
  "num_attention_heads": 40,
  "num_hidden_layers": 64,
  "num_key_value_heads": 8,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bi

11/08/2024 13:55:05 - INFO - llamafactory.data.template - Replace eos token: <|im_end|>


[INFO|configuration_utils.py:672] 2024-11-08 13:55:05,619 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2.5-32B-Instruct-bnb-4bit/snapshots/36adcc220ce3f5747a2ac662a8c11e7236aedc27/config.json
[INFO|configuration_utils.py:739] 2024-11-08 13:55:05,620 >> Model config Qwen2Config {
  "_name_or_path": "unsloth/Qwen2.5-32B-Instruct-bnb-4bit",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 27648,
  "max_position_embeddings": 32768,
  "max_window_layers": 70,
  "model_type": "qwen2",
  "num_attention_heads": 40,
  "num_hidden_layers": 64,
  "num_key_value_heads": 8,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bi

11/08/2024 13:55:05 - WARNING - llamafactory.model.model_utils.quantization - `quantization_bit` will not affect on the PTQ-quantized models.
11/08/2024 13:55:05 - INFO - llamafactory.model.model_utils.quantization - Loading ?-bit BITSANDBYTES-quantized model.
11/08/2024 13:55:05 - INFO - llamafactory.model.patcher - Using KV cache for faster generation.


[WARNING|quantization_config.py:400] 2024-11-08 13:55:05,733 >> Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
[INFO|modeling_utils.py:3726] 2024-11-08 13:55:05,948 >> loading weights file model.safetensors from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2.5-32B-Instruct-bnb-4bit/snapshots/36adcc220ce3f5747a2ac662a8c11e7236aedc27/model.safetensors.index.json
[INFO|modeling_utils.py:1622] 2024-11-08 13:55:05,953 >> Instantiating Qwen2ForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1099] 2024-11-08 13:55:05,954 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151645
}

[INFO|quantizer_bnb_4bit.py:122] 2024-11-08 13:55:06,199 >> target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[INFO|modeling_utils.py:4568] 2024-11-08 13:55:11,826 >> All model checkpoint weights were used when initializing Qwen2ForCausalLM.

[INFO|modeling_utils.py:4576] 2024-11-08 13:55:11,827 >> All the weights of Qwen2ForCausalLM were initialized from the model checkpoint at unsloth/Qwen2.5-32B-Instruct-bnb-4bit.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Qwen2ForCausalLM for predictions without further training.
[INFO|configuration_utils.py:1054] 2024-11-08 13:55:11,962 >> loading configuration file generation_config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--Qwen2.5-32B-Instruct-bnb-4bit/snapshots/36adcc220ce3f5747a2ac662a8c11e7236aedc27/generation_config.json
[INFO|configuration_utils.py:1099] 2024-11-08 13:55:11,963 >> Generate config GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penal

11/08/2024 13:55:12 - INFO - llamafactory.model.model_utils.attention - Using torch SDPA for faster training and inference.
11/08/2024 13:55:13 - INFO - llamafactory.model.adapter - Loaded adapter(s): /Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/saved_models/comics35_pg_nb_Qwen2.5-32B-Instruct-bnb-4bit
11/08/2024 13:55:13 - INFO - llamafactory.model.loader - all params: 32,830,985,216


In [17]:
model.engine

In [18]:
model.engine_type

'huggingface'

In [19]:
with open(test_dataset_file, "r+") as fh:
    test_dataset = json.load(fh)

test_prompts = []
test_grounds = []

for sample in test_dataset:
    test_prompts.append(sample["instruction"] + sample["input"])
    test_grounds.append(sample["output"])

In [20]:
test_predictions = []

In [21]:
messages = []

for prompt in tqdm(test_prompts, desc="Running inferences ..."):
    #print(type(prompt))
    #messages.append({"role": "user", "content": prompt})
    message = [{"role": "user", "content": prompt}]
    test_predictions.append(model.chat(message))

Running inferences ...: 100%|██████████| 156/156 [21:36<00:00,  8.31s/it]


[WARNING|logging.py:328] 2024-11-08 13:55:14,601 >> Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [22]:
len(test_predictions)

156

In [23]:
test_predictions

[[Response(response_text='{"page_utterance_emotions": [["joy"], ["surprise"], ["joy"], ["joy"], ["joy"], ["joy"], ["concern", "joy"], ["joy"], ["joy"], ["joy"], ["joy"]]}', response_length=48, prompt_length=392, finish_reason='stop')],
 [Response(response_text='{"page_utterance_emotions": [["neutral"], ["neutral"], ["anger"], ["anger"], ["neutral"], ["anger", "sadness"], ["anger", "sadness"]]}', response_length=39, prompt_length=534, finish_reason='stop')],
 [Response(response_text='{"page_utterance_emotions": [["anger", "disgust"], ["anger", "disgust"], ["anger", "disgust"], ["anger", "surprise"], ["anger", "surprise"], ["surprise"], ["anger", "surprise"], ["anger"], ["anger"], ["joy"], ["anger"], ["anger"], ["anger"], ["fear"], ["joy"], ["fear", "surprise"]]}', response_length=92, prompt_length=636, finish_reason='stop')],
 [Response(response_text='{"page_utterance_emotions": [["anger", "fear"], ["anger", "fear"], ["anger", "fear"], ["sadness"], ["surprise"], ["neutral"], ["anger"], 

In [24]:
processed_preds = []

for i, raw_pred in enumerate(test_predictions):
    try:
        x = json.loads(raw_pred[0].response_text)["page_utterance_emotions"]
        processed_preds.append(x)
    except:
        print(i)

In [25]:
len(processed_preds)

156

In [26]:
processed_grounds = []

for ground in test_grounds:
    x = json.loads(ground)["page_utterance_emotions"]
    processed_grounds.append(x)

In [27]:
len(processed_grounds)

156

In [28]:
bad_idx = []

for idx, (i,j) in enumerate(zip(processed_grounds, processed_preds)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

132 11 10


In [29]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del processed_grounds[idx]
    del processed_preds[idx]

In [30]:
len(processed_grounds), len(processed_preds)

(155, 155)

In [31]:
grounds = [item for sublist in processed_grounds for item in sublist]
predictions = [item for sublist in processed_preds for item in sublist]

In [32]:
len(grounds), len(predictions)

(1315, 1315)

In [33]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [34]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['concern', 'trust'] will be ignored
  warnings.warn(


In [35]:
y_true_mhot.shape, y_pred_mhot.shape

((1315, 7), (1315, 7))

In [36]:
class_labels = mlb.classes_

In [37]:
class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [38]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.526     0.493     0.509       450
     disgust      0.194     0.240     0.214        50
        fear      0.481     0.438     0.459       297
         joy      0.580     0.610     0.595       290
     neutral      0.486     0.321     0.387       109
     sadness      0.522     0.588     0.553       342
    surprise      0.593     0.583     0.588       355

   micro avg      0.528     0.520     0.524      1893
   macro avg      0.483     0.468     0.472      1893
weighted avg      0.528     0.520     0.522      1893
 samples avg      0.546     0.537     0.515      1893



/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [39]:
# with open({Path(FT_DIR)} / "classification_report.pickle", 'wb') as fh:
    
#      pickle.dump(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3, output_dict=True), fh)